### Crytocurrencies analyis

#### Objective
Show statistical analysis on
- Random variable 'winning', where X='BTC' is the event where BTC price increase the most in pct compared to other ccys during in interval
- Random varible 'no trade', where X='BTC' is the event where BTC was not traded during an interval

#### Approach
Just explore data by counting for now 

#### Some techinical note

In [90]:
# Non tech settings
analysis_interval = 20 # seconds
analysis_period_start = '2017-08-01T00:00:00Z'
analysis_period_end = '2017-08-01T00:02:00Z'
ccys = ['BTC', 'ETH']

In [2]:
# Tech
import os
import sys
import pandas as pd
import numpy as np
import pendulum

module_path = os.path.join('..')
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.db import PgDb
from src.models import DeltaPoint, Trade
from src.computed import compute_price_changes

db = PgDb()

In [91]:
data = [db.get_trades(ccy, analysis_period_start, analysis_period_end) for ccy in ccys]


In [4]:
raw_trades = [Trade(pendulum.instance(t[0]), t[1], 0, 0) for t in data]
trades = sorted(raw_trades, key=lambda x: x.time)c

In [5]:
all(trades[i].time <= trades[i+1].time for i in range(len(trades) - 1))

True

In [38]:
type(pendulum.instance(data[0][0]))
type(raw_trades[0].time)
type(trades[0].time.start_of)

method

In [51]:
from src.models import DeltaPoint
import pendulum

def compute_price_changes(start_point, trades, interval_seconds):
    if interval_seconds not in [15, 20, 30, 60]:
        raise ValueError('Interval value not supported. Must be either 15, 20, 30, or 60')
    if not trades:
        raise ValueError('Trades cannot be empty')
    if not all(trades[i].time <= trades[i+1].time for i in range(len(trades) - 1)):
        raise ValueError('Trades must be in increasing order by time')

    if not start_point:
        first_trade = trades[0]
        first_trade_time = first_trade.time
        start_time = find_closest_possible_start_time(first_trade_time, interval_seconds)
        interval_trades = [t for t in trades if t.time <= start_time]
        start_point = DeltaPoint(start_time, interval_trades[-1].price, 0, 0, 0)

    result = []
    while True:
        next_time = start_point.time.add(seconds=interval_seconds)
        if next_time > trades[-1].time:
            break

        interval_trades = [t for t in trades if t.time > start_point.time and t.time <= next_time]
        if not interval_trades:
            next_delta = DeltaPoint(next_time, start_point.price, 0, 0, 0)
        else:
            last_trade = interval_trades[-1]
            d = last_trade.price - start_point.price
            pct = d / start_point.price * 100
            n = len(interval_trades)
            next_delta = DeltaPoint(next_time, last_trade.price, d, pct, n)

        result.append(next_delta)
        start_point = next_delta

    return result

def find_closest_possible_start_time(timestamp, interval):
    ''' Find closest possible start time given a timestamp and interval
    E.g.
        - Given timestamp: 2017-08-01T00:01:24
        - Given interval: 20s

        Start time CAN'T be 
        - 2017-08-01T00:01:00, or 
        - 2017-08-01T00:01:20
        since we won't have data for this

        The closest possible start time is then 2017-08-01T00:01:40
    
    Arguments:
        timestamp: pendulum 
        interval: integer
    '''
    next_start_time = timestamp.start_of('minute')
    while True:
        if next_start_time >= timestamp:
            return next_start_time

        next_start_time = next_start_time.add(seconds=interval)


In [53]:
result = compute_price_changes(None, trades, 20)
print(result)

[DeltaPoint(time=<Pendulum [2017-08-01T00:00:20+03:00]>, price=Decimal('2882.7000000000'), d=Decimal('2.7000000000'), pct=Decimal('0.0937500'), n=23), DeltaPoint(time=<Pendulum [2017-08-01T00:00:40+03:00]>, price=Decimal('2883.0000000000'), d=Decimal('0.3000000000'), pct=Decimal('0.01040691018836507440940784681'), n=21), DeltaPoint(time=<Pendulum [2017-08-01T00:01:00+03:00]>, price=Decimal('2883.0000000000'), d=Decimal('0E-10'), pct=Decimal('0'), n=14), DeltaPoint(time=<Pendulum [2017-08-01T00:01:20+03:00]>, price=Decimal('2883.5000000000'), d=Decimal('0.5000000000'), pct=Decimal('0.01734304543877904960110995491'), n=8), DeltaPoint(time=<Pendulum [2017-08-01T00:01:40+03:00]>, price=Decimal('2879.6000000000'), d=Decimal('-3.9000000000'), pct=Decimal('-0.1352522975550546211201664644'), n=20)]


In [86]:
result_as_series = {
    'time': [dp.time.isoformat() for dp in result],
    'price': [float(dp.price) for dp in result],
    'price_delta': [float(dp.d) for dp in result],
    'price_delta_pct': [float(dp.pct) for dp in result],
    'number_of_trades': [float(dp.n) for dp in result],
}


In [88]:
pd.set_option('display.precision', 8)
df = pd.DataFrame(result_as_series)
df.head()

,time,price,price_delta,price_delta_pct,number_of_trades
0,2017-08-01T00:00:20+03:00,2882.7,2.7,0.09375000,23.0
1,2017-08-01T00:00:40+03:00,2883.0,0.3,0.01040691,21.0
2,2017-08-01T00:01:00+03:00,2883.0,0.0,0.00000000,14.0
3,2017-08-01T00:01:20+03:00,2883.5,0.5,0.01734305,8.0
4,2017-08-01T00:01:40+03:00,2879.6,-3.9,-0.13525230,20.0


In [ ]:
df = pd.DataFrame(trades, columns=['Time', 'Price'])
df.head()

In [ ]:
df['Time'].max()

In [ ]:
df.tail()

In [9]:
from src.models import DeltaPoint
import pendulum

def compute_price_changes(start_point, trades, interval_seconds):
    if interval_seconds not in [15, 20, 30, 60]:
        raise ValueError('Interval value not supported. Must be either 15, 20, 30, or 60')
    if not trades:
        raise ValueError('Trades cannot be empty')
    if not all(trades[i].time <= trades[i+1].time for i in range(len(trades) - 1)):
        raise ValueError('Trades must be in increasing order by time')

    if not start_point:
        first_trade = trades[0]
        first_trade_time = first_trade[0].time
        start_time = find_closest_possible_start_time(first_trade_time, interval_seconds)
        interval_trades = [t for t in trades if t.time <= start_time]
        start_point = DeltaPoint(start_time, interval_trades[-1].price, 0, 0, 0)

    result = []
    while True:
        next_time = start_point.time.add(seconds=interval_seconds)
        if next_time > trades[-1].time:
            break

        interval_trades = [t for t in trades if t.time > start_point.time and t.time <= next_time]
        if not interval_trades:
            next_delta = DeltaPoint(next_time, start_point.price, 0, 0, 0)
        else:
            last_trade = interval_trades[-1]
            d = last_trade.price - start_point.price
            pct = d / start_point.price * 100
            n = len(interval_trades)
            next_delta = DeltaPoint(next_time, last_trade.price, d, pct, n)

        result.append(next_delta)
        start_point = next_delta

    return result

def find_closest_possible_start_time(timestamp, interval):
    ''' Find closest possible start time given a timestamp and interval
    E.g.
        - Given timestamp: 2017-08-01T00:01:24
        - Given interval: 20s

        Start time CAN'T be 
        - 2017-08-01T00:01:00, or 
        - 2017-08-01T00:01:20
        since we won't have data for this

        The closest possible start time is then 2017-08-01T00:01:40
    
    Arguments:
        timestamp: pendulum 
        interval: integer
    '''

    next_start_time = timestamp.start_of('minute')
    while True:
        if next_start_time >= timestamp:
            return next_start_time

        next_start_time = next_start_time.add(seconds=interval)
